In [1]:
import gc
import time
import pandas as pd
import numpy as np
import csv
import pywt
import os
from sklearn.metrics import f1_score,hamming_loss
from sklearn.metrics import precision_recall_fscore_support as prf
import warnings
warnings.filterwarnings("ignore")
path = "/media/jdcloud/"

In [2]:
def preprocess_y(labels, y, num_class=10):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

In [100]:
train = []

#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold1.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold2.csv").values)
path = './'

# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold2.csv").values) #1fold f0.822

#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold2.csv").values) # 1fold f0.813

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold1.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold2.csv").values)

train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold0.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold2.csv").values) # 3folds f0.817

train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_f0819_10net_fold.csv").values) # 1fold f0.819

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold2.csv").values) # 3folds f0.817

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold2.csv").values)

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold1.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold2.csv").values)

In [101]:
test = []

#test.append(pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold0.csv").values)
#test.append(pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold1.csv").values)
#test.append( pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold2.csv").values)

#test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold0.csv").values)
#test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold1.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold2.csv").values)

test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold0.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold2.csv").values)

test.append(pd.read_csv(path+"test_csv/"+"densenet_f0819_10net_fold.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_sym_addori_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_sym_addori_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_sym_addori_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_db6_addori_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_db6_addori_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_db6_addori_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_ori_addori_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_ori_addori_fold1.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_10net_ori_addori_fold2.csv").values)

In [5]:
# import lightgbm as lgb
# # specify your configurations as a dict
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',#'multiclass',
#     'metric': 'binary_logloss',
#     'num_class': 1,
#     'num_leaves': 100,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0,
#     'seed':2019
# }

In [107]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet, Lasso, RidgeCV,LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from mlknn import MLkNN
from mltsvm import MLTSVM

gbm = GradientBoostingClassifier(learning_rate=0.005,n_estimators=100,max_depth=5,min_samples_leaf=20,
                                min_samples_split=600,subsample=0.7,random_state=2019)
LR = LogisticRegression(penalty="l2",C=1.0)
#Eln = ElasticNet()
#Las = Lasso(alpha=0.2)
#LRR = RidgeCV()

pre_type = "db6"#"db6"#"sym"
labels = pd.read_csv("/media/uuser/data/final_codes/final_run_test/REFERENCE.csv")

index = np.arange(6689)
y_train = preprocess_y(labels, index)

x_train = np.hstack(train)

#train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.333, random_state=0)
#train = lgb.Dataset(x_train,label=y_train)
#valid = lgb.Dataset(valid_x, label=valid_y
#gbm = lgb.train(params,train,num_boost_round=1000,#valid_sets=valid,early_stopping_rounds=5)
LR1 = LinearRegression()
clf = MLkNN(k=10)
#clf = MLTSVM(c_k=0.5)
clf = OneVsRestClassifier(gbm)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=20, min_sa...      subsample=0.7, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          n_jobs=None)

In [104]:
# out = np.hstack(test)#
# out0 = np.hstack(train[:-1])#
# out1 = np.hstack(train[:-2])
# out2 = np.hstack(train[:-2]+[train[-1]])

# LR_clf = joblib.load("LR_ensemble0810.pkl")
# MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")
# MLkNN10_clf = joblib.load("MLkNN10_ensemble.pkl")
# MLkNN10_db6_clf = joblib.load("MLkNN10_ensemble_db6_0810.pkl")

# y_pred_LR = LR_clf.predict(out0)
# y_pred_proba_LR = LR_clf.predict_proba(out0)

# y_pred_MLkNN8 = MLkNN8_clf.predict(out1).toarray()
# y_pred_proba_MLkNN8 = MLkNN8_clf.predict_proba(out1).toarray()

# y_pred_MLkNN10 = MLkNN10_clf.predict(out1).toarray()
# y_pred_proba_MLkNN10 = MLkNN10_clf.predict_proba(out1).toarray()

# y_pred_MLkNN10_db6 = MLkNN10_db6_clf.predict(out2).toarray()
# y_pred_proba_MLkNN10_db6 = MLkNN10_db6_clf.predict_proba(out2).toarray()

# y_pred_LR[:,2] = y_pred_MLkNN8[:,2]
# y_pred_proba_LR[:,2] = y_pred_proba_MLkNN8[:,2]

# y_pred_LR[:,5] = y_pred_MLkNN8[:,5]
# y_pred_proba_LR[:,5] = y_pred_proba_MLkNN8[:,5]

# y_pred_LR[:,6] = y_pred_MLkNN8[:,6]
# y_pred_proba_LR[:,6] = y_pred_proba_MLkNN8[:,6]

# y_pred_LR[:,9] = y_pred_MLkNN8[:,9]
# y_pred_proba_LR[:,9] = y_pred_proba_MLkNN8[:,9]

# y_pred_LR[:,4] = y_pred_MLkNN10[:,4]
# y_pred_proba_LR[:,4] = y_pred_proba_MLkNN10[:,4]

# y_pred_LR[:,8] = y_pred_MLkNN10[:,8]
# y_pred_proba_LR[:,8] = y_pred_proba_MLkNN10[:,8]

# y_pred_LR[:,7] = y_pred_MLkNN10_db6[:,7]
# y_pred_proba_LR[:,7] = y_pred_proba_MLkNN10_db6[:,7]


# y_pred = y_pred_LR
# y_pred_proba_train = y_pred_proba_LR

y_pred = clf.predict(x_train)#.toarray()
y_pred_proba_train = clf.predict(x_train)#.toarray()
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))
for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.9438611559218751
f1 score of ab 0 is 0.970402878458721
f1 score of ab 1 is 0.9927127363990341
f1 score of ab 2 is 0.9688245635681747
f1 score of ab 3 is 0.987408533509226
f1 score of ab 4 is 0.9593696850761612
f1 score of ab 5 is 0.9713568647666131
f1 score of ab 6 is 0.9613769768078739
f1 score of ab 7 is 0.9616455422371305
f1 score of ab 8 is 0.9552754993527292
f1 score of ab 9 is 0.9529752533087663
 train data hamming_loss  : 0.011989834055912692
 train data precision recall f1  : (0.9469527084267703, 0.9477450540688693, 0.9434733469716277, None)


In [ ]:
 train data f1_score  : 0.9503375188836033
f1 score of ab 0 is 0.9720396461822378
f1 score of ab 1 is 0.9904887607485836
f1 score of ab 2 is 0.9677540961794703
f1 score of ab 3 is 0.986531683978493
f1 score of ab 4 is 0.9623842989623262
f1 score of ab 5 is 0.9790668350732303
f1 score of ab 6 is 0.9730658300240653
f1 score of ab 7 is 0.964883553430314
f1 score of ab 8 is 0.9585124067148397
f1 score of ab 9 is 0.9623326213834353
 train data hamming_loss  : 0.010913439976080132
 train data precision recall f1  : (0.9574550256640254, 0.9600039866447402, 0.9554396882996076, None)


In [105]:
out = np.hstack(test)#
# out0 = np.hstack(test[:-1])#
# out1 = np.hstack(test[:-2])
# out2 = np.hstack(test[:-2]+[test[-1]])

# LR_clf = joblib.load("LR_ensemble0810.pkl")
# MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")
# MLkNN10_clf = joblib.load("MLkNN10_ensemble.pkl")
# MLkNN10_db6_clf = joblib.load("MLkNN10_ensemble_db6_0810.pkl")

# y_pred_LR = LR_clf.predict(out0)
# y_pred_proba_LR = LR_clf.predict_proba(out0)

# y_pred_MLkNN8 = MLkNN8_clf.predict(out1).toarray()
# y_pred_proba_MLkNN8 = MLkNN8_clf.predict_proba(out1).toarray()

# y_pred_MLkNN10 = MLkNN10_clf.predict(out1).toarray()
# y_pred_proba_MLkNN10 = MLkNN10_clf.predict_proba(out1).toarray()

# y_pred_MLkNN10_db6 = MLkNN10_db6_clf.predict(out2).toarray()
# y_pred_proba_MLkNN10_db6 = MLkNN10_db6_clf.predict_proba(out2).toarray()

# y_pred_LR[:,2] = y_pred_MLkNN8[:,2]
# y_pred_proba_LR[:,2] = y_pred_proba_MLkNN8[:,2]

# y_pred_LR[:,5] = y_pred_MLkNN8[:,5]
# y_pred_proba_LR[:,5] = y_pred_proba_MLkNN8[:,5]

# y_pred_LR[:,6] = y_pred_MLkNN8[:,6]
# y_pred_proba_LR[:,6] = y_pred_proba_MLkNN8[:,6]

# y_pred_LR[:,9] = y_pred_MLkNN8[:,9]
# y_pred_proba_LR[:,9] = y_pred_proba_MLkNN8[:,9]

# y_pred_LR[:,4] = y_pred_MLkNN10[:,4]
# y_pred_proba_LR[:,4] = y_pred_proba_MLkNN10[:,4]

# y_pred_LR[:,8] = y_pred_MLkNN10[:,8]
# y_pred_proba_LR[:,8] = y_pred_proba_MLkNN10[:,8]

# y_pred_LR[:,7] = y_pred_MLkNN10_db6[:,7]
# y_pred_proba_LR[:,7] = y_pred_proba_MLkNN10_db6[:,7]


# y_pred = y_pred_LR
# y_pred_proba = y_pred_proba_LR

y_pred = clf.predict(out)#.toarray()
y_pred_proba = clf.predict_proba(out)#.toarray()

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

pred = []
for j in range(y_pred.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])

''' ''' 
for i, val in enumerate(pred):
    if val == []:
        pass
        #for i_p, val_p in enumerate(y_pred_proba[i]):
        #    if val_p >= 0.4:
        #        pred[i].append(i_p)    # f1 == 0.832
                
        if y_pred_proba[i][np.argmax(y_pred_proba[i])] >= 0.4:
            pred[i] = [np.argmax(y_pred_proba[i])]     # f1 == 0.833  0.4
path = "/media/jdcloud"
val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0809.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

AttributeError: 'LinearRegression' object has no attribute 'predict_proba'

In [71]:
from sklearn.externals import joblib
# Save
joblib.dump(clf,"MLkNN10_ensemble_db6_0810.pkl")#LR_ensemble

['MLkNN10_ensemble_db6_0810.pkl']

In [466]:
thr = 0.6
#out1 = thr * (0.1 * train[0] + 0.3 * train[1] + 0.6 * train[2])
#out2 = (1 - thr) * (1.0 * train[3])
thr0 = 0; thr1 = 0.4; thr2 = 0.; thr3 = 0.0; thr4 = 0.; thr5 = 0.6;
train_y = thr0*train[0] + thr1*train[1] + thr2*train[2] + thr3*train[3] + thr4*train[4] + thr5*train[5]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))# 'micro', 'weighted'

best_threshold : [0.7 0.5 0.5 0.4 0.6 0.2 0.3 0.4 0.5 0.3]
 train data f1_score  : 0.9226479088215832
f1 score of ab 0 is 0.9576243119144202
f1 score of ab 1 is 0.9794953051205606
f1 score of ab 2 is 0.951682745472497
f1 score of ab 3 is 0.982355844958436
f1 score of ab 4 is 0.9477985152051004
f1 score of ab 5 is 0.9721827424838709
f1 score of ab 6 is 0.958814994471973
f1 score of ab 7 is 0.9415063228088438
f1 score of ab 8 is 0.9398197061954244
f1 score of ab 9 is 0.9297211830048123
 train data hamming_loss  : 0.016474809388548364
 train data precision recall f1  : (0.9262096975133303, 0.9349429411471569, 0.9257665392364153, None)


In [137]:
best_threshold

array([0.6, 0.6, 0.5, 0.5, 0.6, 0.2, 0.2, 0.7, 0.5, 0.4])

In [395]:
out = thr0 * test[0]# + thr1 * test[1] + thr2 * test[2]# + thr3 * test[3] + thr4 * test[4] + thr5 * test[5]

y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])
    
for i, val in enumerate(pred):
    ''' 
    if 0 in val and len(val) > 1:
        flag = 0
        for j in val:
            if (test_y[i][0] - best_threshold[0]) > (test_y[i][j] - best_threshold[j]):
                pass
            else:
                flag = 1
        if flag == 1:
            pred[i] = val[1:]
        else:
            pred[i] = val[0]
    '''
    if len(val) == 0:
        pred[i] = [np.argmin(np.abs(best_threshold - out_test[i]))]
        
val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0803.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

# 0.833 and 0.842 ensemble 

In [334]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty="l2",C=1.0)


pre_type = "db6"#"db6"#"sym"
labels = pd.read_csv(path + "REFERENCE.csv")

index = np.arange(6689)
y_train = preprocess_y(labels, index)

x_train = np.hstack([train_y,y_pred_proba_train])

clf = OneVsRestClassifier(LR)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_train)

print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))
for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

 train data f1_score  : 0.9211840609081451
f1 score of ab 0 is 0.9603084096823336
f1 score of ab 1 is 0.9793386425479889
f1 score of ab 2 is 0.9513702179183157
f1 score of ab 3 is 0.9822910749256628
f1 score of ab 4 is 0.9444799508628059
f1 score of ab 5 is 0.9696737288321967
f1 score of ab 6 is 0.9584407222873685
f1 score of ab 7 is 0.9350094053104944
f1 score of ab 8 is 0.9451176555412449
f1 score of ab 9 is 0.930177490596017


In [333]:
out = 0.4 * test[0] + 0.6 * test[1]
x_test = np.hstack([out,y_pred_proba])
y_pred = clf.predict(x_test)
y_pred_proba = clf.predict_proba(x_test)

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

pred = []
for j in range(y_pred.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])

''' ''' 
for i, val in enumerate(pred):
    if val == []:
        pass
        #for i_p, val_p in enumerate(y_pred_proba[i]):
        #    if val_p >= 0.4:
        #        pred[i].append(i_p)    # f1 == 0.832
                
        if y_pred_proba[i][np.argmax(y_pred_proba[i])] >= 0.4:
            pred[i] = [np.argmax(y_pred_proba[i])]     # f1 == 0.833


val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0806.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

# quarter-final data test

In [16]:
quarter_labels= pd.read_csv("/media/uuser/data/final_codes/final_run_semi/reference.csv")
quarter_index = np.arange(6500)
quarter_y_train = preprocess_y(quarter_labels, quarter_index)

In [99]:
quarter_train = []
path = "./"
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_attention_10net_fold2.csv").values) 

# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold0.csv").values)
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold2.csv").values)

#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold0.csv").values)
#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold0.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_sym_addori_fold2.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_db6_addori_fold2.csv").values)

''' '''
# out = np.hstack(test)#
out0 = np.hstack(quarter_train[:-1])#
out1 = np.hstack(quarter_train[:-2])
out2 = np.hstack(quarter_train[:-2]+[quarter_train[-1]])

# LR_clf = joblib.load("LR_ensemble0810.pkl")
# MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")
# MLkNN10_clf = joblib.load("MLkNN10_ensemble.pkl")
# MLkNN10_db6_clf = joblib.load("MLkNN10_ensemble_db6_0810.pkl")

y_pred_LR = LR_clf.predict(out0)
y_pred_proba_LR = LR_clf.predict_proba(out0)

y_pred_MLkNN8 = MLkNN8_clf.predict(out1).toarray()
y_pred_proba_MLkNN8 = MLkNN8_clf.predict_proba(out1).toarray()

y_pred_MLkNN10 = MLkNN10_clf.predict(out1).toarray()
y_pred_proba_MLkNN10 = MLkNN10_clf.predict_proba(out1).toarray()

y_pred_MLkNN10_db6 = MLkNN10_db6_clf.predict(out2).toarray()
y_pred_proba_MLkNN10_db6 = MLkNN10_db6_clf.predict_proba(out2).toarray()

y_pred_LR[:,2] = y_pred_MLkNN8[:,2]
# y_pred_proba_LR[:,2] = y_pred_proba_MLkNN8[:,2]

y_pred_LR[:,5] = y_pred_MLkNN8[:,5]
# y_pred_proba_LR[:,5] = y_pred_proba_MLkNN8[:,5]

y_pred_LR[:,6] = y_pred_MLkNN8[:,6]
# y_pred_proba_LR[:,6] = y_pred_proba_MLkNN8[:,6]

y_pred_LR[:,9] = y_pred_MLkNN8[:,9]
# y_pred_proba_LR[:,9] = y_pred_proba_MLkNN8[:,9]

y_pred_LR[:,4] = y_pred_MLkNN10[:,4]
# y_pred_proba_LR[:,4] = y_pred_proba_MLkNN10[:,4]

y_pred_LR[:,8] = y_pred_MLkNN10[:,8]
# y_pred_proba_LR[:,8] = y_pred_proba_MLkNN10[:,8]

y_pred_LR[:,7] = y_pred_MLkNN10_db6[:,7]
# y_pred_proba_LR[:,7] = y_pred_proba_MLkNN10_db6[:,7]


quarter_y_pred = y_pred_LR
# quarter_y_pred_proba = y_pred_proba_LR

# quarter_x_train = np.hstack(quarter_train)
# quarter_y_pred = clf.predict(quarter_x_train).toarray()

print(" train data f1_score  :", f1_score(quarter_y_train, quarter_y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(quarter_y_train[:, i], quarter_y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(quarter_y_train, quarter_y_pred)) 
print(" train data precision recall f1  :", prf(quarter_y_train, quarter_y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.8439097952017833
f1 score of ab 0 is 0.971215859308423
f1 score of ab 1 is 0.9877104179896463
f1 score of ab 2 is 0.9606090069979423
f1 score of ab 3 is 0.9621594687564023
f1 score of ab 4 is 0.9481268620677086
f1 score of ab 5 is 0.9751797264318665
f1 score of ab 6 is 0.9664972199135371
f1 score of ab 7 is 0.9524740975127066
f1 score of ab 8 is 0.9526784074522501
f1 score of ab 9 is 0.49884348496530456
 train data hamming_loss  : 0.01376923076923077
 train data precision recall f1  : (0.9443076923076923, 0.9599820512820513, 0.9466612942612942, None)


In [496]:
Counter(quarter_y_train[:,7])

Counter({0: 6276, 1: 224})

In [497]:
from collections import Counter

Counter(quarter_y_pred[:,7])

Counter({0: 6276, 1: 224})

In [524]:
(0.843 * 10 - 0.498 + 0.8) / 10

0.8732

In [ ]:
 train data f1_score  : 0.9226479088215832
f1 score of ab 0 is 0.9576243119144202
f1 score of ab 1 is 0.9794953051205606
f1 score of ab 2 is 0.951682745472497
f1 score of ab 3 is 0.982355844958436
f1 score of ab 4 is 0.9477985152051004
f1 score of ab 5 is 0.9721827424838709
f1 score of ab 6 is 0.958814994471973
f1 score of ab 7 is 0.9415063228088438
f1 score of ab 8 is 0.9398197061954244
f1 score of ab 9 is 0.9297211830048123